In [1]:
import logging
from typing import Tuple

from fastai.vision.all import *

In [2]:


logging.basicConfig(filename='mnist.log', level=logging.INFO)
def custom_logger(text):
	logging.info(text)	
	print(text)



Downloaded Data
Loaded Data
Number of Images : Threes -> 6131, Sevens -> 6265
Converted into Tensors...
An image has a size of : torch.Size([784])
Built Dataset & Dataloaders
| Epoch 0 |
Train Loss : 1063.31, Valid Accuracy : 95.39%

| Epoch 1 |
Train Loss : 378.21, Valid Accuracy : 96.25%

| Epoch 2 |
Train Loss : 327.99, Valid Accuracy : 96.57%

| Epoch 3 |
Train Loss : 304.01, Valid Accuracy : 96.52%

| Epoch 4 |
Train Loss : 288.77, Valid Accuracy : 96.72%



In [3]:
class Dataset:
	def __init__(self, data, labels, split=None, shuffle=True):
		if isinstance(data, list): data = tensor(data)
		if isinstance(labels, list): labels = tensor(labels)    
            
		if data.shape[0] != labels.shape[0]:
			raise ValueError("The data and labels shapes don't match")
            
		if shuffle is True:
			indexes = torch.randperm(data.shape[0])
			data = data[indexes]
			labels = labels[indexes]
            
		if split:
			split_int = int(data.shape[0] * split)    
			self.train = Dataset(data[:split_int], labels[:split_int])
			self.valid = Dataset(data[split_int:], labels[split_int:])
		
		self.data = data
		self.labels = labels
		
	def __getitem__(self, key):
		return (self.data[key],self.labels[key])

	def __iter__(self):
		return iter((self.data, self.labels))

	def __len__(self):
		return self.data.shape[0] # could any of them
    
def Dataloader(ds, bs=100):
    return [ds[pos:pos + bs] for pos in range(0, len(ds), bs)]

In [4]:
# Forward Pass Funcs
def sigmoid(z):
    return 1 / (1 + torch.exp(-z))

def linear1(x, w, b):
    return x@w + b

def predict(x:torch.Tensor, w, b) -> Tuple[torch.Tensor, torch.Tensor]:
    x1 = linear1(x, w, b)
    res = sigmoid(x1)
    return (res > .5).float(), res #labels, probs

In [5]:
# Loss & Metric
def mnist_loss(y, ypred) -> torch.Tensor:
    return torch.where(y == 1., 1.-ypred, ypred).sum()

def accuracy(y, ypred) -> float:
    return (y == ypred).float().mean()


In [6]:
# Training Procedures
def init_params(size):
    w = torch.randn(size).requires_grad_()
    b = torch.randn(1).requires_grad_()
    return (w, b)

def optimize(loss, w, b, lr):
    loss.backward()
    w.data -= lr*w.grad
    b.data -= lr*b.grad
    w.grad = None
    b.grad = None
    
def validate_model(w, b, valid_dl):
    with torch.no_grad():
        val_loss = tensor(0.)
        val_accs = []
        for x, y in valid_dl:
            labels, yprobs = predict(x, w, b)
            val_loss += mnist_loss(y, yprobs) # Batch Loss
            val_accs.append(accuracy(y, labels)) # Batch Acc

    return val_loss.item(), torch.stack(val_accs).mean().item() #Overall Loss, Acc

In [ ]:
def cross-entropy-loss()